# Pycryptodome 설치하기

- Pycrypto (Python Cryptography Toolkit)은 SHA256과 같은 해시 함수와 AES, DES, RSA등과 같은 다양한 알고리즘을 제공하는 파이썬 패키지
- Pycryptodome 는 Pycrypto 를 대체한 새로운 패키지 중 하나임


In [1]:
# 패키지 설치
!pip install pycryptodome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.6 MB/s eta 0:00:00


**패키지 로드 해보기**

In [ ]:
import Crypto

# 3DES 구현하기
- DES 는 1970년대 IBM 에서 파이스텔 블록 구조에 기반하여 설계되고 개발된 56비트키 암호화 알고리즘
- DES를 보완하여 대체한 것이 3DES (Triple DES) 인데, 3DES는 암호화를 위해 블록당 3번의 DES를 수행함
- 56*3 = 168 비트키 암호화 알고리즘이며 DES의 핵심 알고리즘은 변경 없이 적용됨

## 8글자로 된 문장 암호화
- 3DES CBC 모드로 암호화 수행 //연쇄
- key text 가 짧더라도 원하는 길이의 암호 키를 얻을 수 있도록 SHA256 해시 함수 사용
- Python 3에서 모든 문자열은 유니코드임
- 3DES 는 16바이트 혹은 24바이트의 크기의 키를 사용 (실습에서는 24바이트 크기의 키를 사용할 예정)
- 3DES는 64비트 암호화 블록 크기를 가짐


In [ ]:
from Crypto.Cipher import DES3 #사이퍼에서 그중 DES 사용한다
from Crypto.Hash import SHA256 as SHA 

In [ ]:
keytext = 'testtes'#짧은 키 텍스트로 24바이트 생성, 몇 글자여도 상관x

hash_fn = SHA.new()#해시 function 생성
hash_fn.update(keytext.encode('utf-8'))#hash function.update 후 키를 utf-8파일로 인코딩
print(keytext.encode('utf-8'))#앞에 b가 붙은것은 바이트타입이라는 것
key = hash_fn.digest()[:24]#24바이트 생성

b'testtes'


In [ ]:
type(key)

bytes

In [ ]:
print(len(key))
print(key)

24
b"\xe3\xb0\xc4B\x98\xfc\x1c\x14\x9a\xfb\xf4\xc8\x99o\xb9$'\xaeA\xe4d\x9b\x93L"


In [ ]:
#내가 한번 키 생성 해볼래, SHA사용해서
#keytext="testtes"
#hash_f=SHA.new()
#hash_f.update(keytext.encode('utf-8'))
#key=hash_f.digest()[:24]
#intext="init"
#hash_f.update(intext.encode('utf-8'))
#iv=hash_f.digest[:8]
#msg="python37"
#des3=DES3.new(key,DES3.MODE_CBC,iv) //객체생성
#enc_msg=des3.encrypt(msg.encode());
#des3=DES3.new(key,DES3.MODE_CBC.iv)
#dec_msg=des3.decrypt(enc_msg)
#dec_msg.decode()

**초기화벡터**
- 초기화 벡터를 생성하는 데에도 SHA256 해시 이용
- 입력 문자를 넣으면, 이를 이용해서 SHA256 해시를 이용해 초기화 벡터 생성

In [ ]:
ivtext = 'init'
hash_fn.update(ivtext.encode('utf-8'))
iv = hash_fn.digest()[:8]#8바이트의 이니셜 value 얻음


In [ ]:
type(iv)

bytes

**암호화**
- plaintext 에 담긴 문자열을 3DES로 암호화
- DES3 를 생성하기 위해서 '암호키', '운영모드', '초기화 벡터' 를 인자로 넣어줌
- 운영 모드에 따라서 초기화 벡터가 필요 없을 수도 있음. 예를들어, ECB 모드(전자코드북)는 초기화 벡터가 필요 없음
- Python 의 string 은 유니코드 이므로 encode() 함수를 이용해서 인코딩 된 유니코드 문자열의 bytes를 반환

In [ ]:
msg = "python37"#8바이트에 맞춰 넣음

In [ ]:
print(msg.encode())#byte타입변환

b'python37'


In [ ]:
des3 = DES3.new(key, DES3.MODE_CBC, iv)#key, CBC모드 설정, 초기화벡터 ->객체
encmsg = des3.encrypt(msg.encode())#encode 필수, 안그러면 암호화X
print(encmsg)

b'S\x93\x1b\xfbw\xadk\x10'


**복호화**
- 복호화시 주의할 점은 위에 암호화시에 만들어 놓은 DES3 객체를 재사용하지 않고, 복호화를 위한 객체를 각각 생성하고 활용해야 함

In [ ]:
des3 = DES3.new(key, DES3.MODE_CBC, iv)#똑같이 만들어야함. enc직후 dec못해서 재생성
decmsg = des3.decrypt(encmsg)

In [ ]:
print(msg)
print(encmsg)
print(type(decmsg))
print(decmsg.decode())

I love Python#I love Python2##I love Python3
b'S\x93\x1b\xfbw\xadk\x10'
<class 'bytes'>
python37


In [ ]:
msg2 = "python375"#9글자는 오류남, 8의 배수만 가능
des3 = DES3.new(key, DES3.MODE_CBC, iv)
enc_msg2 = des3.encrypt(msg2.encode())

ValueError: ignored

## 메세지 블록 패딩
- 블록 암호에서는 데이터가 블록 크기와 같아야 함
- 임의의 길이의 메세지를 암호화 하기 위해서는 메세지를 블록 크기에 맞게 패딩하고, 운영 모드를 활용하여 다수의 블록에 대한 암호화를 수행함
- 위에서 블록 크기가 64비트 이므로 8바이트 블록 크기에 맞도록 메세지의 변환이 필요


In [ ]:
def make8String(msg):#8글자로 맞춰주는 함수
  msglen = len(msg)
  filler = ""
  if msglen%8 != 0:
    filler = '0'*(8-msglen%8)#'0'*7 하여 8의 배수로 맞추어 줌
  msg += filler
  return msg

In [ ]:
msg2 = make8String(msg2)
print(msg2)
des3 = DES3.new(key, DES3.MODE_CBC, iv)
enc_msg2 = des3.encrypt(msg2.encode())
print(enc_msg2)
print(len(enc_msg2))
des3_ = DES3.new(key, DES3.MODE_CBC, iv)
dec_msg2 = des3_.decrypt(enc_msg2)
print(dec_msg2)

python3750000000
b'S\x93\x1b\xfbw\xadk\x10\xe5\xb4\xfd\x8b7\xa8U\x04'
16
b'python3750000000'


**데이터 무결성 보장**
- 위의 복호화는 원래의 msg 에 '0'이 추가되어 복호화된 메세지가 길어지게 됨. 이는 사실 원문 메세지가 변형된 것임
- 원문이 만약 텍스트가 아니라 바이너리로 되어 있는 정보일 경우에는 복호화한 정보의 길이가 원래 정보와 다르다면, 이는 복호화된 파일이 위조 또는 변조 되었을 수 있고, 악성 코드가 숨겨져 있을 수도 있음
- 암호화된 정보를 복호화 했을 경우, 반드시 원래 정보와 동일하다는 것을 보장해야 문제가 없음
- 추가된 문자 '0'을 제거하여 원래 정보로 복원하는 것으로 이러한 문제 해결
- 원래 정보에 추가된 '0'의 개수에 대한 정보를 포함시켜서 암호화 수행
- 추가된 '0'의 개수를 포함시키기 위한 앞에 header block 을 추가함

In [ ]:
def makeEnabled(msg):
  fillersize = 0
  textsize = len(msg)
  if textsize %8 !=0:
    fillersize = 8-textsize%8
  filler = '0'*fillersize
  header = '%d' %(fillersize) # str(fillersize)
  gap = 8-len(header)
  header += '#'*gap
  return header+msg+filler


In [ ]:
msg2 = "python375wd0"
msg2 = makeEnabled(msg2)
print(msg2)
print(len(msg2))
des3 = DES3.new(key, DES3.MODE_CBC, iv)
enc_msg2 = des3.encrypt(msg2.encode())

4#######python375wd00000
24


In [ ]:
des3 = DES3.new(key,DES3.MODE_CBC, iv)
dec_msg2=des3.decrypt(enc_msg2)#enc_msg2.encode() 안하는이유는 이미 byte라서
print(enc_msg2)
print(dec_msg2)

b'\xbe\x94\x82`Y2\x7f\x99nA\xadu\xe2\xf2\xd6\xc5\xc5eP\xb3:\xf2#\x99'
b'4#######python375wd00000'


**메세지 채움 제거**
- 블록 크기를 맞추기 위해서 메세지 채움을 수행했을 경우 해당 암호문을 복호화 한 후에는 채움을 위한 부분들을 제거하여 원래 문자열이 되도록 하는 후처리 과정이 필요
- 이를 위해 헤더 블록의 # 앞 부분에 채움 문자 개수를 확인하고 이 개수 만큼의 뒤에 채움 문자를 제거한 문자열을 후처리 과정을 통해 얻을 수 있음
- split 함수: 문자열 객체 메소드인 split() 은 인자로 입력된 문자 또는 문자 열을 구분자로 해서 문자열을 분리하고 순서대로 리스트에 담고 이 리스트를 리턴함


In [ ]:
# split 함수 예제
msg = 'I love Python#I love Python2##I love Python3'
print(msg.split('#')[0])# 이 #을 기준으로 분류 할 수 있다, [0]= 맨 앞 부분
#위에서는 4#######python375wd00000를 split 이용

I love Python


In [ ]:
def postDecrypt(msg):
  header = msg[:8].decode()#encode의 반대 의미, byte->string
  fillersize = int(header.split('#')[0])
  if fillersize !=0:
    postmsg = msg[8:-fillersize]#0~7까진 header블록이고, 8블록에서 -filersize만큼 빼고 출력
  else:
    postmsg = msg[8:]
  return postmsg.decode()


In [ ]:
print(msg2)
postmsg = postDecrypt(msg2.encode())
print(postmsg)

4#######python375wd00000
python375wd0


# 클래스 활용하기
- 파이썬은 c++ 처럼 객체지향 프로그래밍이 가능한 언어로 클래스를 정의하여 함수와 동일한 역할을 하는 메소드와 변수 역할을 하는 멤버로 구성됨
- 메소드나 멤버는 클래스에 속하여 정의된 것으로 이름만 다를 뿐이지 일반적인 함수나 변수와 동일함
- 클래스를 잘 활용하면 간결하고 체계적인 프로그래밍을 할 수 있음
- 재사용 가능성이 높은 코드는 클래스로 구현하여 여러 모듈에서 활용될 수 있도록 하면 효율적인 프로그래밍이 가능

**클래스 정의하기**

클래스를 정의하는 방법은 다음과 같음

-----
class 클래스 이름 (부모 클래스 이름):
  
  클래스 생성자

  클래스 소멸자
  
  멤버 정의
  
  메소드 정의

  

-----

- 클래스는 상속이 가능한 이름 공간 (name space)로 상속을 받는 클래스를 자식 클래스, 상속을 하는 클래스를 부모 클래스라고 부름. 
- 부모 클래스가 없을 경우에는 클래스 정의에서 괄호 속의 부모 클래스 이름 생략 가능
- 멤버 정의: 클래스 멤버는 self.멤버이름 으로 정의함 (self는 자기 자신을 의미). 클래스 멤버는 클래스 내에서 전역 변수와 같이 사용됨. 클래스 멤버는 메소드 내에서 정의될 수도 있고, 클래스 생성 시에 정의하고 싶을 경우에는 생성자 내에서 정의됨
- 메소드 정의: 클래스 메소드는 일반 함수와 같은 방법으로 정의하되 첫번째 인자는 반드시 self 이어야 함
- 정의된 메소들을 클래스 안에서 호출하는 경우** self.메소드**와 같이 self 를 메소드 이름 앞에 붙여서 호출하고, 멤버를 사용할 경우에도 self.멤버 와 같이 self 를 멤버 앞에 붙여 사용함
- 클래스 생성자: 클래스의 객체가 생성될 때 자동으로 실행되는 클래스 메소드
- 클래스 소멸자: 클래스의 인스턴스 객체가 소멸될 때 자동적으로 실행되는 클래서 메소드





In [ ]:
## 예시
class myClass:#(부모클래스): 생략중
  
  # 클래스 생성자
  def __init__(self):# def __init__(self) -->생성자, self를 반드시 첫번째 인자로 넣어야 한다, 
  #추가적으로 받아야하는 인자가 있다면 해당하는 변수 명시하는건데 self 지우고 명시인것인지 모르겠음
  # 위에 나온 sha.new(키, 모드, iv)도 일종의 생성자
    print('make class instance')
    # 멤버 정의
    self.name = 'my name'#생성자에서 my name 멤버가 정의됨

  # 클래스 소멸자
  def __del__(self):
    print('delete class instance')
  
  # 메소드 정의
  def return_name(self):
    print(self.name)#self.name이 전역변수처럼 사용하네? 를 알 수 있음
    return self.name
  


In [ ]:
testclass = myClass()

make class instance


In [ ]:
print(testclass.name)
testclass.name = "saerom"
print(testclass.name)

my name
saerom


In [ ]:
aa = testclass.return_name()
print(aa)#리턴값 aa

saerom
saerom


In [ ]:
del testclass

delete class instance


In [ ]:
testclass

NameError: ignored

# AES 클래스 구현하기
- AES 는 키 크기가 (128, 192, 256) 이 가능하고, 암호화 블록 크기가 128 비트인 부분만 빼고 다른 부분들은 3DES 와 비슷
- AES를 클래스 타입으로 지정하여 원하는 크기의 키를 얻기 위한 keytext를 입력하고, CBC 모드에서 128비트의 초기 상태를 얻기 위한 ivtext 를 입력 받아 이로부터 암호화 및 복호화를 수행
- 클래스 내부 함수로 암호화, 복호화, 메세지 채움, 복호화된 메세지 후처리 등의 함수들을 정의


In [ ]:
from Crypto.Cipher import AES
from Crypto.Hash import SHA256 as SHA

In [ ]:
class myAES():
  # 클래스 생성자
  def __init__(self, keytext, ivtext, keysize = 128):
    hash_fn = SHA.new()
    hash_fn.update(keytext.encode('utf-8'))
    key = hash_fn.digest()#256비트, 32바이트 생성하기 때문에
    keylen = int(keysize/8)#16바이트만 필요하기 때문에 16바이트길이만 알아내서
    self.key = key[:keylen]#여기서 16바이트만 잘라냄

    hash_fn.update(ivtext.encode('utf-8'))
    iv = hash_fn.digest()
    self.iv = iv[:16] # 블록 크기가 128비트 이므로 초기화 벡터가 16 바이트 이어야 함

  # 메소드 1: 메세지 채움
  def makeEnabled(self, plaintext):#여기서 알 수 있듯, self는 지워지지 않음
    fillersize = 0
    textsize = len(plaintext)
    # 블록 크기가 128비트 이므로 블록 크기에 맞지 않을 경우 메세지 채움 수행
    if textsize %16 != 0: 
      fillersize = 16-textsize%16

    filler = '0'*fillersize
    header = '%d' %(fillersize)
    gap = 16-len(header)
    header += '#'*gap

    return header+plaintext+filler

  # 메소드 2: 메세지 암호화
  def encrypt(self, plaintext):
    plaintext = self.makeEnabled(plaintext)
    aes = AES.new(self.key, AES.MODE_CBC, self.iv)
    encmsg = aes.encrypt(plaintext.encode())
    return encmsg

  # 메소드 3: 복호화 메세지 후처리
  def postDec(self, dec_msg):
    header = dec_msg[:16].decode()#[:16] 안써도 틀리진 않지만, 쓰면 더 명확하다
    fillersize = int(header.split('#')[0])
    if fillersize !=0:
      decmsg = dec_msg[16:-fillersize]
    else:
      decmsg = dec_msg[16:]
    return decmsg.decode()

  # 메소드 4: 암호문 복호화
  def decrypt(self, ciphertext):
    aes = AES.new(self.key, AES.MODE_CBC, self.iv)
    dec_msg = aes.decrypt(ciphertext)

    return self.postDec(dec_msg)
    #return dec_msg



In [ ]:
myaes = myAES('keytxt','iniue')

In [ ]:
msg = 'other message'

In [ ]:
enc_msg = myaes.encrypt(msg)

In [ ]:
dec_msg = myaes.decrypt(enc_msg)

In [ ]:
print(msg)
print(enc_msg)
print(dec_msg)

other message
b'\xc18\x17\xb4\x96\x89\xd9#\x86\x0e]\x1f\x8a|\x88u\xd0\xd8\x1d\x82\x1e\xaf\xb3\xff4%z\xe1q\xa8\xd0)'
other message


In [ ]:
myaes#myAES의 클래스다 라고 출력된 것임

# 메세지 블록 패딩 (2)
위에 메세지 블록 패딩에서는 문자열을 이용해서 '0' 이라는 문자로 패딩을 수행하였지만, 수업 시간에는 마지막 블록이 완전히 채워질 때까지 평문에 여분의 바이트들을 추가하는 채우기 기법에 대해서 다루었음. 
- 원래 메세지는 복호화 후에 채워진 값을 통해 복원할 수 있음
- 채움 비트가 어디까지인지는 블록의 마지막 바이트를 통해 알 수 있음



In [ ]:
def makeEnabled(msg):
  length = 16-(len(msg)%16)
  msg += chr(length)*length

  return msg

In [ ]:
print(makeEnabled("i lobe ydnasdnk"))
print(len(makeEnabled("pythontest")))
print(makeEnabled("pythontestsalkdsads").encode())#06이 encode해야 보임

i lobe ydnasdnk
16
b'pythontestsalkdsads\r\r\r\r\r\r\r\r\r\r\r\r\r'


In [ ]:
print(type(makeEnabled("pythontest")))
print(type(makeEnabled("pythontest").encode()))

<class 'str'>
<class 'bytes'>


In [ ]:
print(makeEnabled("pythontest").encode().decode())
makeEnabled("pythontest").encode().decode()

pythontest


'pythontest\x06\x06\x06\x06\x06\x06'

In [ ]:
print(len("pythontest"))

10


In [ ]:
makeEnabled("pythontest")[-1]
#int(makeEnabled("pythontest")[-1])

'\x06'

In [ ]:
int(makeEnabled("pythontest").encode()[-1])

6

In [ ]:
print(makeEnabled("pythontest").encode()[:-6])

b'pythontest'


# 새 섹션

In [2]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import base64

# Generate a random 256-bit key
key = get_random_bytes(32)

# Initialize the cipher with the key and mode (AES-256 in CBC mode)
cipher = AES.new(key, AES.MODE_CBC)

# The data to be encrypted
data = b'This is a secret message'

# Pad the data to match the block size of AES (16 bytes)
padded_data = pad(data, AES.block_size)

# Encrypt the padded data
encrypted_data = cipher.encrypt(padded_data)

# Encode the encrypted data in base64 for storage or transmission
encoded_data = base64.b64encode(encrypted_data)

# Decrypt the encoded data

# Decode the base64 encoded data
decoded_data = base64.b64decode(encoded_data)

# Initialize the cipher with the same key and mode
decipher = AES.new(key, AES.MODE_CBC, iv=cipher.iv)

# Decrypt the data and remove the padding
decrypted_data = unpad(decipher.decrypt(decoded_data), AES.block_size)

# Print the decrypted data
print(decrypted_data.decode('utf-8'))


This is a secret message


In [3]:
import hashlib
import os

# Generate a random salt
salt = os.urandom(16)

# Create a new SHA-256 hash object
sha256_hash = hashlib.sha256()

# Data to be hashed
data = b'This is some data to be hashed'

# Add the salt to the data
salted_data = salt + data

# Update the hash object with the salted data
sha256_hash.update(salted_data)

# Get the hexadecimal representation of the hash digest
hash_digest = sha256_hash.hexdigest()

# Print the hash digest
print(hash_digest)


67b94b110037afb792bf36d1e8cfb336308cff2c42d180a01dda120b0fb12f83
